# 판다스로 검색, 질문 챗봇만들기

In [22]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2 tiktoken langchain_openai langchain_experimental

ERROR: Cannot install grobid-client==0.7.1, openai==1.6.1, openai==1.7.0 and openai==1.7.1 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [9]:
import openai
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

from dotenv import load_dotenv
load_dotenv()

True

## 엑셀, CSV 검색, Aggregation

In [ ]:
import pandas as pd

df = pd.read_csv("https://github.com/kairess/toy-datasets/raw/master/titanic.csv")

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [26]:
!pip install langchain_experimental

  Obtaining dependency information for langchain_experimental from https://files.pythonhosted.org/packages/01/61/9d8a9ed481bf245ca5b678c211488e1b51c7c4e5441a5923ee8a4eee248c/langchain_experimental-0.0.49-py3-none-any.whl.metadata
  Using cached langchain_experimental-0.0.49-py3-none-any.whl.metadata (1.9 kB)
  Obtaining dependency information for langchain<0.2,>=0.1 from https://files.pythonhosted.org/packages/23/98/c70fac0f1b3193ced86013b563119c27c68ac26b684815f407555224108d/langchain-0.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 18.8 MB/s eta 0:00:00m eta 0:00:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.354
    Uninstalling langchain-0.0.354:
      Successfully uninstalled langchain-0.0.354
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"), df, verbose=True)

agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To find the number of rows in the dataframe, I can use the `shape` attribute of the dataframe.
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 891
Thought:I now know the final answer
Final Answer: There are 891 rows in the dataframe.

> Finished chain.


'There are 891 rows in the dataframe.'

In [28]:
agent.run("행이 몇 개지?")



> Entering new AgentExecutor chain...
Thought: To find the number of rows in the dataframe, I can use the `shape` attribute of the dataframe.

Action: python_repl_ast
Action Input: df.shape[0]
Observation: 891
Thought:The number of rows in the dataframe is 891.
Final Answer: 891

> Finished chain.


'891'

In [29]:
agent.run("승객들의 평균 연령은?")



> Entering new AgentExecutor chain...
Thought: To find the average age of the passengers, I need to calculate the mean of the 'Age' column in the dataframe.

Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 29.69911764705882
Thought:The average age of the passengers is 29.69911764705882.
Final Answer: 29.69911764705882

> Finished chain.


'29.69911764705882'

In [33]:
agent.run("남성과 여성의 비율은?")



> Entering new AgentExecutor chain...
Thought: To find the ratio of males to females, I need to count the number of males and females in the dataframe.

Action: Use the `value_counts()` method on the 'Sex' column of the dataframe.

Action Input: `df['Sex'].value_counts()`

Observation: Use the `value_counts()` method on the 'Sex' column of the dataframe. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the `python_repl_ast` tool to execute the command.

Action: `python_repl_ast`

Action Input: `df['Sex'].value_counts()`

Observation: `python_repl_ast` is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the `python_repl_ast` tool to execute the command.

Action: `python_repl_ast`

Action Input: `df['Sex'].value_counts()`

Observation: `python_repl_ast` is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the `python_repl_ast` tool to execute the command.

Action: `python_repl_ast`

Action Input: `df['Sex'].value_counts()

'Agent stopped due to iteration limit or time limit.'

In [34]:
agent.run("객실 등급과 성별에 따른 생존자 수를 계산해줘")



> Entering new AgentExecutor chain...
Thought: I need to group the data by "Pclass" and "Sex" columns and then calculate the sum of the "Survived" column for each group.
Action: python_repl_ast
Action Input: df.groupby(["Pclass", "Sex"])["Survived"].sum()
Observation: Pclass  Sex   
1       female    91
        male      45
2       female    70
        male      17
3       female    72
        male      47
Name: Survived, dtype: int64
Thought:I now know the number of survivors for each combination of passenger class and sex.
Final Answer: The number of survivors for each combination of passenger class and sex are as follows:
- Pclass 1, female: 91
- Pclass 1, male: 45
- Pclass 2, female: 70
- Pclass 2, male: 17
- Pclass 3, female: 72
- Pclass 3, male: 47

> Finished chain.


'The number of survivors for each combination of passenger class and sex are as follows:\n- Pclass 1, female: 91\n- Pclass 1, male: 45\n- Pclass 2, female: 70\n- Pclass 2, male: 17\n- Pclass 3, female: 72\n- Pclass 3, male: 47'